In [7]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
LangChain 记忆对话系统演示
展示不同类型的记忆机制在对话中的应用

包含以下记忆类型：
1. ConversationBufferMemory - 基础缓冲记忆
2. ConversationBufferWindowMemory - 窗口缓冲记忆
3. ConversationSummaryMemory - 摘要记忆
4. ConversationSummaryBufferMemory - 摘要缓冲记忆
5. ConversationTokenBufferMemory - 基于token的缓冲记忆
"""

import os
import asyncio
from typing import Dict, List, Any, Optional
from datetime import datetime

# LangChain imports
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
    ConversationSummaryBufferMemory,
    ConversationTokenBufferMemory,
    ChatMessageHistory
)
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain_ollama import OllamaLLM  # 使用新的导入
from langchain_openai import ChatOpenAI

# 设置环境变量（如果需要使用OpenAI）
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

print("LangChain 记忆对话系统演示")
print("=" * 50)


LangChain 记忆对话系统演示


In [8]:
# ===================== 配置部分 =====================

# 模型配置
OLLAMA_BASE_URL = "http://localhost:11434"
OLLAMA_MODEL = "gemma3:4b"  # 或者其他已安装的模型

# 创建LLM实例
def create_llm(use_openai: bool = False):
    """
    创建语言模型实例

    Args:
        use_openai: 是否使用OpenAI模型

    Returns:
        LLM实例
    """
    if use_openai:
        # 使用OpenAI模型
        return ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=0.7,
            max_tokens=1000
        )
    else:
        # 使用本地Ollama模型
        return OllamaLLM(
            base_url=OLLAMA_BASE_URL,
            model=OLLAMA_MODEL,
            temperature=0.7
        )

# 创建LLM实例（默认使用Ollama）
llm = create_llm(use_openai=False)

print(f"使用模型: {OLLAMA_MODEL}")
print(f"Ollama地址: {OLLAMA_BASE_URL}")


使用模型: gemma3:4b
Ollama地址: http://localhost:11434


In [5]:
# ===================== 1. ConversationBufferMemory 演示 =====================

print("\n1. ConversationBufferMemory - 基础缓冲记忆")
print("-" * 40)

# 创建基础缓冲记忆
buffer_memory = ConversationBufferMemory(
    return_messages=True,  # 返回消息对象而不是字符串
    memory_key="history"  # 记忆在prompt中的键名，使用默认的"history"
)

# 创建自定义prompt模板以确保兼容性
from langchain.prompts import PromptTemplate

buffer_prompt = PromptTemplate(
    input_variables=["history", "input"],
    template="""以下是人类和AI之间的友好对话。AI很健谈，并根据其上下文提供大量具体细节。如果AI不知道问题的答案，它会诚实地说不知道。

当前对话:
{history}
人类: {input}
AI:"""
)

# 创建对话链
conversation_buffer = ConversationChain(
    llm=llm,
    memory=buffer_memory,
    prompt=buffer_prompt,
    # verbose=True  # 显示详细信息
)

print("ConversationBufferMemory 特点:")
print("- 保存完整的对话历史")
print("- 记忆会随着对话增长而增长")
print("- 适合短对话或记忆容量充足的场景")

# 模拟对话
def demo_buffer_memory():
    """演示基础缓冲记忆"""
    print("\n开始对话演示...")

    # 第一轮对话
    response1 = conversation_buffer.predict(input="你好，我叫张三，我是一名软件工程师")
    print(f"用户: 你好，我叫张三，我是一名软件工程师")
    print(f"AI: {response1}")

    # 第二轮对话
    response2 = conversation_buffer.predict(input="我喜欢编程和阅读技术书籍")
    print(f"\n用户: 我喜欢编程和阅读技术书籍")
    print(f"AI: {response2}")

    # 第三轮对话 - 测试记忆
    response3 = conversation_buffer.predict(input="你还记得我的名字和职业吗？")
    print(f"\n用户: 你还记得我的名字和职业吗？")
    print(f"AI: {response3}")

    # 查看记忆内容
    print(f"\n当前记忆内容:")
    print(f"记忆变量: {buffer_memory.memory_key}")
    print(f"记忆内容: {buffer_memory.chat_memory.messages}")

# 运行演示
demo_buffer_memory()



1. ConversationBufferMemory - 基础缓冲记忆
----------------------------------------
ConversationBufferMemory 特点:
- 保存完整的对话历史
- 记忆会随着对话增长而增长
- 适合短对话或记忆容量充足的场景

开始对话演示...
用户: 你好，我叫张三，我是一名软件工程师
AI: 你好，张三！很高兴认识你。作为一名软件工程师，你可能从事各种各样的项目吧？ 我注意到软件工程是一个非常广泛的领域，涵盖了从前端开发（例如使用React、Angular或Vue.js构建用户界面）到后端开发（例如使用Node.js、Python或Java构建服务器端逻辑）再到数据库管理（例如MySQL、PostgreSQL或MongoDB）以及 DevOps 实践（例如使用Docker、Kubernetes和AWS）。

你更专注于哪方面？你主要使用哪些编程语言和技术栈？  或者，你目前正在处理什么类型的项目？ 告诉我更多关于你的工作内容，我很高兴了解！


用户: 我喜欢编程和阅读技术书籍
AI: AI: 哇，编程和阅读技术书籍，这真是个好爱好！这表明你对技术有持续的学习兴趣，这在软件工程领域至关重要。你喜欢阅读什么样的技术书籍呢？ 是关于算法和数据结构，或者面向对象编程，还是更偏向于特定编程语言的深入指南？

我注意到，技术书籍通常会深入探讨特定的技术领域。例如，像“算法导论”这样的书会让你掌握排序、搜索和动态规划等核心算法，而像“深入理解 JavaScript”这样的书则会让你对 JavaScript 的内部机制有更全面的理解。 

另外，你可能也对设计模式感兴趣，比如工厂模式、单例模式等等。 这些模式可以帮助你编写更易于维护和扩展的代码。

你最近读过哪些技术书籍？ 或者，你最喜欢的编程书籍有哪些？ 告诉我一些你觉得特别有价值的书籍，我很乐意和您讨论！ 另外，你最喜欢用哪种编程语言呢？ 比如，C++、Python、Java、JavaScript？


用户: 你还记得我的名字和职业吗？
AI: AI: 当然记得！你的名字是张三，是一名软件工程师。很高兴再次见到你！你对软件工程的兴趣和对技术书籍的热爱让我对你有了更深入的了解。现在，你有什么想进一步探讨的吗？例如，你最近在学习什么新的技术，

In [9]:
# ===================== 2. ConversationBufferWindowMemory 演示 =====================

print("\n\n2. ConversationBufferWindowMemory - 窗口缓冲记忆")
print("-" * 50)

# 创建窗口缓冲记忆（只保留最近2轮对话）
window_memory = ConversationBufferWindowMemory(
    k=2,  # 保留最近2轮对话
    return_messages=True,
    memory_key="history"  # 使用默认的"history"
)

# 创建对话链
conversation_window = ConversationChain(
    llm=llm,
    memory=window_memory,
    prompt=buffer_prompt,  # 使用相同的prompt模板
    max_tokens_limit=100,  # 设置最大token数
    # verbose=True
)

print("ConversationBufferWindowMemory 特点:")
print("- 只保留最近k轮对话")
print("- 记忆大小固定，不会无限增长")
print("- 适合长对话场景")
print(f"- 当前设置: k={window_memory.k}")

def demo_window_memory():
    """演示窗口缓冲记忆"""
    print("\n开始对话演示...")

    # 多轮对话
    conversations = [
        "我叫李四，是一名数据科学家",
        "我在北京工作",
        "我喜欢机器学习和深度学习",
        "我最近在研究大语言模型",
        "你还记得我的名字吗？",  # 这时应该忘记了最初的信息
        "你还记得我在哪里工作吗？"  # 这个信息可能还记得
    ]

    for i, user_input in enumerate(conversations, 1):
        response = conversation_window.predict(input=user_input)
        print(f"\n第{i}轮对话:")
        print(f"用户: {user_input}")
        print(f"AI: {response}")

        # 显示当前窗口内容
        messages = window_memory.chat_memory.messages
        print(f"当前窗口大小: {len(messages)//2} 轮对话")

    print(f"\n最终记忆内容 (最近{window_memory.k}轮):")
    for msg in window_memory.chat_memory.messages:
        role = "用户" if isinstance(msg, HumanMessage) else "AI"
        print(f"{role}: {msg.content}")

# 运行演示
demo_window_memory()




2. ConversationBufferWindowMemory - 窗口缓冲记忆
--------------------------------------------------
ConversationBufferWindowMemory 特点:
- 只保留最近k轮对话
- 记忆大小固定，不会无限增长
- 适合长对话场景
- 当前设置: k=2

开始对话演示...


C:\Users\34769\AppData\Local\Temp\ipykernel_35128\298349334.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  window_memory = ConversationBufferWindowMemory(



第1轮对话:
用户: 我叫李四，是一名数据科学家
AI: 好的李四，很高兴认识你！作为一名数据科学家，这真是个非常有趣且充满挑战的领域。 

让我先简单介绍一下我自己。我是一个大型语言模型，由 Google 训练。我的核心功能是理解和生成人类语言。我所掌握的信息量非常庞大，涵盖了从历史、科学、文化到艺术等各个方面。

作为数据科学家，你可能经常接触到各种数据分析、建模和可视化任务。你可能使用 Python、R 这样的编程语言，以及像 Pandas、Scikit-learn、TensorFlow 这样的工具。  你可能也需要处理各种数据格式，比如 CSV、JSON、SQL 数据库等等。 

你对哪些具体的领域感兴趣呢？ 例如，你主要从事什么类型的项目？  你是侧重于预测分析、机器学习、自然语言处理，还是其他方面？ 告诉我你的工作内容，我可以尝试提供更多相关的知识和信息。

当前窗口大小: 1 轮对话

第2轮对话:
用户: 我在北京工作
AI: 好的，李四，在北京工作真是太棒了！北京作为中国的政治、文化和科技中心，数据科学领域也发展非常活跃。

考虑到你在北京工作，我能提供一些关于北京数据科学领域的信息：

*   **主要行业:** 北京的数据科学应用领域非常广泛，包括但不限于：
    *   **金融:** 许多银行和金融机构都在北京进行大数据分析、风险管理、欺诈检测等工作。例如，中国工商银行、招商银行等都有大型数据科学团队。
    *   **科技:** 百度、腾讯、阿里巴巴等巨头公司在北京都有大量的研发中心和数据科学团队，主要从事搜索引擎优化、推荐系统、广告投放等方向。
    *   **政府:** 北京市政府也大力推动大数据治理，利用数据分析来提升城市管理水平，例如交通优化、环境监测、公共安全等方面。
    *   **医疗:** 北京拥有清华大学和北京大学等顶尖学府，在生物医学大数据分析、基因组学、新药研发等方面都有着很强的实力。
    *   **消费电子:**  华策方舟、小米等公司在北京都有着大量的数据科学团队，主要从事用户行为分析、产品优化、市场营销等方向。

*   **常用技术栈:** 鉴于你在北京工作，我推测你可能使用以下技术栈：
    *   **编程语言:** Python (尤其 Pandas, Scikit-learn)

KeyboardInterrupt: 

In [ ]:
# ===================== 3. ConversationSummaryMemory 演示 =====================

print("\n\n3. ConversationSummaryMemory - 摘要记忆")
print("-" * 45)

# 创建摘要记忆
summary_memory = ConversationSummaryMemory(
    llm=llm,  # 用于生成摘要的LLM
    return_messages=True,
    memory_key="history"  # 使用默认的"history"
)

# 创建对话链
conversation_summary = ConversationChain(
    llm=llm,
    memory=summary_memory,
    prompt=buffer_prompt,  # 使用相同的prompt模板
    # verbose=True
)

print("ConversationSummaryMemory 特点:")
print("- 将历史对话总结为摘要")
print("- 记忆大小相对固定")
print("- 保留重要信息，丢弃细节")
print("- 适合超长对话场景")

def demo_summary_memory():
    """演示摘要记忆"""
    print("\n开始对话演示...")

    # 模拟一段较长的对话
    conversations = [
        "你好，我是王五，今年28岁，是一名产品经理",
        "我在上海的一家互联网公司工作，主要负责移动应用产品",
        "我们公司最近在开发一款社交软件，用户增长很快",
        "我的团队有5个人，包括2名开发工程师、1名设计师、1名测试工程师和我",
        "我们每周都会开产品评审会，讨论新功能和用户反馈",
        "最近我们在考虑加入AI功能，比如智能推荐和聊天机器人",
        "你能总结一下我刚才说的信息吗？"
    ]

    for i, user_input in enumerate(conversations, 1):
        response = conversation_summary.predict(input=user_input)
        print(f"\n第{i}轮对话:")
        print(f"用户: {user_input}")
        print(f"AI: {response}")

        # 显示当前摘要
        if hasattr(summary_memory, 'moving_summary_buffer') and summary_memory.moving_summary_buffer:
            print(f"当前摘要: {summary_memory.moving_summary_buffer}")

    print(f"\n最终摘要内容:")
    print(f"摘要: {summary_memory.moving_summary_buffer}")

# 运行演示
demo_summary_memory()


In [ ]:
# ===================== 4. ConversationSummaryBufferMemory 演示 =====================

print("\n\n4. ConversationSummaryBufferMemory - 摘要缓冲记忆")
print("-" * 55)

# 创建摘要缓冲记忆
summary_buffer_memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,  # 当超过100个token时开始摘要
    return_messages=True,
    memory_key="history"  # 使用默认的"history"
)

# 创建对话链
conversation_summary_buffer = ConversationChain(
    llm=llm,
    memory=summary_buffer_memory,
    prompt=buffer_prompt,  # 使用相同的prompt模板
    verbose=True
)

print("ConversationSummaryBufferMemory 特点:")
print("- 结合摘要和缓冲记忆的优点")
print("- 保留最近的对话 + 历史摘要")
print("- 当缓冲区超过限制时，旧对话被摘要")
print(f"- 当前token限制: {summary_buffer_memory.max_token_limit}")

def demo_summary_buffer_memory():
    """演示摘要缓冲记忆"""
    print("\n开始对话演示...")

    conversations = [
        "我是赵六，是一名AI研究员，专注于自然语言处理",
        "我在清华大学获得了计算机科学博士学位",
        "我发表了多篇关于Transformer模型的论文",
        "目前我在一家AI公司担任首席科学家",
        "我们团队正在开发下一代对话AI系统",
        "这个系统将具备更强的推理和记忆能力",
        "你能告诉我关于我的背景信息吗？"
    ]

    for i, user_input in enumerate(conversations, 1):
        response = conversation_summary_buffer.predict(input=user_input)
        print(f"\n第{i}轮对话:")
        print(f"用户: {user_input}")
        print(f"AI: {response}")

        # 显示记忆状态
        print(f"摘要部分: {summary_buffer_memory.moving_summary_buffer}")
        print(f"缓冲区消息数: {len(summary_buffer_memory.chat_memory.messages)}")

# 运行演示
demo_summary_buffer_memory()


In [ ]:
# ===================== 5. ConversationTokenBufferMemory 演示 =====================

print("\n\n5. ConversationTokenBufferMemory - Token缓冲记忆")
print("-" * 50)

# 创建基于token的缓冲记忆
token_buffer_memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=150,  # 最大token数量
    return_messages=True,
    memory_key="history"  # 使用默认的"history"
)

# 创建对话链
conversation_token_buffer = ConversationChain(
    llm=llm,
    memory=token_buffer_memory,
    prompt=buffer_prompt,  # 使用相同的prompt模板
    verbose=True
)

print("ConversationTokenBufferMemory 特点:")
print("- 基于token数量限制记忆")
print("- 自动删除最旧的消息以保持在限制内")
print("- 更精确的内存控制")
print(f"- 当前token限制: {token_buffer_memory.max_token_limit}")

def demo_token_buffer_memory():
    """演示基于token的缓冲记忆"""
    print("\n开始对话演示...")

    conversations = [
        "我是钱七，一名创业者，正在开发一款教育科技产品",
        "我们的产品使用AI技术为学生提供个性化学习体验",
        "目前我们已经获得了天使轮投资，团队规模达到15人",
        "我们的用户主要是中小学生，覆盖数学、英语、科学等科目",
        "我们计划在明年推出更多功能，包括虚拟导师和学习分析",
        "你还记得我是做什么的吗？"
    ]

    for i, user_input in enumerate(conversations, 1):
        # 计算当前token数量
        current_tokens = token_buffer_memory.llm.get_num_tokens_from_messages(
            token_buffer_memory.chat_memory.messages
        )

        response = conversation_token_buffer.predict(input=user_input)
        print(f"\n第{i}轮对话:")
        print(f"用户: {user_input}")
        print(f"AI: {response}")
        print(f"对话前token数: {current_tokens}")
        print(f"当前消息数: {len(token_buffer_memory.chat_memory.messages)}")

# 运行演示
demo_token_buffer_memory()


In [ ]:
# ===================== 6. 自定义记忆类演示 =====================

print("\n\n6. 自定义记忆类 - 带有用户信息提取的记忆")
print("-" * 55)

from langchain.memory.chat_memory import BaseChatMemory
from langchain.schema import BaseMessage
from pydantic import BaseModel
import json

class UserProfileMemory(BaseChatMemory, BaseModel):
    """
    自定义记忆类，专门提取和存储用户个人信息
    """
    user_profile: Dict[str, Any] = {}
    conversation_history: List[BaseMessage] = []
    max_history_length: int = 10

    class Config:
        arbitrary_types_allowed = True

    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """保存对话上下文并提取用户信息"""
        # 保存到历史记录
        input_str = inputs.get(self.input_key, "")
        output_str = outputs.get(self.output_key, "")

        self.chat_memory.add_user_message(input_str)
        self.chat_memory.add_ai_message(output_str)

        # 提取用户信息（简单的关键词匹配）
        self._extract_user_info(input_str)

        # 限制历史长度
        messages = self.chat_memory.messages
        if len(messages) > self.max_history_length * 2:  # *2 因为每轮对话有两条消息
            self.chat_memory.messages = messages[-(self.max_history_length * 2):]

    def _extract_user_info(self, text: str):
        """从文本中提取用户信息"""
        text_lower = text.lower()

        # 提取姓名
        if "我叫" in text or "我是" in text:
            # 简单的姓名提取逻辑
            import re
            name_pattern = r"我叫([^，,。.]+)|我是([^，,。.]+)"
            match = re.search(name_pattern, text)
            if match:
                name = match.group(1) or match.group(2)
                if name and not any(word in name for word in ["一名", "一个", "在"]):
                    self.user_profile["name"] = name.strip()

        # 提取职业
        job_keywords = ["工程师", "程序员", "经理", "科学家", "研究员", "老师", "医生", "律师", "设计师"]
        for keyword in job_keywords:
            if keyword in text:
                self.user_profile["job"] = keyword
                break

        # 提取年龄
        age_pattern = r"(\d+)岁"
        age_match = re.search(age_pattern, text)
        if age_match:
            self.user_profile["age"] = age_match.group(1)

        # 提取地点
        if "在" in text and ("工作" in text or "生活" in text or "住" in text):
            location_pattern = r"在([^工作生活住，,。.]+)"
            location_match = re.search(location_pattern, text)
            if location_match:
                location = location_match.group(1).strip()
                if len(location) <= 10:  # 避免提取过长的文本
                    self.user_profile["location"] = location

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """加载记忆变量"""
        messages = self.chat_memory.messages

        # 构建记忆内容
        memory_content = ""

        # 添加用户档案信息
        if self.user_profile:
            memory_content += "用户信息:\n"
            for key, value in self.user_profile.items():
                memory_content += f"- {key}: {value}\n"
            memory_content += "\n"

        # 添加最近对话
        if messages:
            memory_content += "最近对话:\n"
            for msg in messages[-6:]:  # 最近3轮对话
                role = "用户" if isinstance(msg, HumanMessage) else "AI"
                memory_content += f"{role}: {msg.content}\n"

        return {self.memory_key: memory_content}

    def clear(self) -> None:
        """清空记忆"""
        super().clear()
        self.user_profile = {}

# 创建自定义记忆实例
custom_memory = UserProfileMemory(
    memory_key="history",  # 使用默认的"history"
    input_key="input",
    output_key="response"
)

# 创建自定义prompt模板
custom_prompt = PromptTemplate(
    input_variables=["history", "input"],  # 修改为"history"
    template="""你是一个友好的AI助手。请根据以下信息与用户对话：

{history}

当前用户输入: {input}
AI回复:"""
)

# 创建对话链
conversation_custom = ConversationChain(
    llm=llm,
    memory=custom_memory,
    prompt=custom_prompt,
    verbose=True
)

def demo_custom_memory():
    """演示自定义记忆"""
    print("\n开始自定义记忆演示...")

    conversations = [
        "你好，我叫孙八，今年32岁",
        "我是一名软件工程师，在深圳工作",
        "我喜欢编程和打篮球",
        "我最近在学习机器学习",
        "你能告诉我你记住了我的哪些信息吗？",
        "我的年龄是多少？",
        "我在哪里工作？"
    ]

    for i, user_input in enumerate(conversations, 1):
        response = conversation_custom.predict(input=user_input)
        print(f"\n第{i}轮对话:")
        print(f"用户: {user_input}")
        print(f"AI: {response}")

        # 显示提取的用户信息
        print(f"提取的用户信息: {custom_memory.user_profile}")

# 运行演示
demo_custom_memory()


In [ ]:
# ===================== 7. 记忆类型比较和选择指南 =====================

print("\n\n7. 记忆类型比较和选择指南")
print("-" * 40)

def compare_memory_types():
    """比较不同记忆类型的特点"""

    memory_comparison = {
        "ConversationBufferMemory": {
            "优点": ["保留完整对话历史", "实现简单", "信息无损失"],
            "缺点": ["内存使用随对话增长", "可能超出模型上下文限制"],
            "适用场景": ["短对话", "内存充足", "需要完整历史"],
            "推荐指数": "⭐⭐⭐"
        },
        "ConversationBufferWindowMemory": {
            "优点": ["内存使用固定", "保留最近对话", "实现简单"],
            "缺点": ["丢失早期信息", "窗口大小需要调优"],
            "适用场景": ["中等长度对话", "关注最近交互", "内存有限"],
            "推荐指数": "⭐⭐⭐⭐"
        },
        "ConversationSummaryMemory": {
            "优点": ["内存使用相对固定", "保留重要信息", "适合长对话"],
            "缺点": ["摘要可能丢失细节", "需要额外LLM调用", "摘要质量依赖模型"],
            "适用场景": ["超长对话", "需要历史概览", "内存严格限制"],
            "推荐指数": "⭐⭐⭐⭐"
        },
        "ConversationSummaryBufferMemory": {
            "优点": ["结合摘要和缓冲优点", "灵活的记忆管理", "平衡细节和概览"],
            "缺点": ["实现复杂", "需要调优参数", "额外LLM调用"],
            "适用场景": ["长对话", "需要平衡记忆", "生产环境"],
            "推荐指数": "⭐⭐⭐⭐⭐"
        },
        "ConversationTokenBufferMemory": {
            "优点": ["精确的token控制", "避免上下文溢出", "自动管理"],
            "缺点": ["需要token计算", "可能突然丢失信息"],
            "适用场景": ["严格token限制", "API成本敏感", "自动化系统"],
            "推荐指数": "⭐⭐⭐⭐"
        }
    }

    for memory_type, info in memory_comparison.items():
        print(f"\n{memory_type}:")
        print(f"  推荐指数: {info['推荐指数']}")
        print(f"  优点: {', '.join(info['优点'])}")
        print(f"  缺点: {', '.join(info['缺点'])}")
        print(f"  适用场景: {', '.join(info['适用场景'])}")

compare_memory_types()


In [ ]:
# ===================== 8. 实际应用示例 - 智能客服系统 =====================

print("\n\n8. 实际应用示例 - 智能客服系统")
print("-" * 45)

class CustomerServiceBot:
    """
    智能客服机器人，使用多种记忆策略
    """

    def __init__(self, llm, memory_type="summary_buffer"):
        self.llm = llm
        self.memory_type = memory_type
        self.memory = self._create_memory()
        self.conversation = self._create_conversation()

        # 客服知识库（简化版）
        self.knowledge_base = {
            "退货": "我们支持7天无理由退货，请提供订单号和退货原因。",
            "发货": "订单确认后24小时内发货，快递3-5个工作日送达。",
            "支付": "支持微信、支付宝、银行卡等多种支付方式。",
            "优惠": "新用户享受首单8折优惠，会员可享受更多折扣。"
        }

    def _create_memory(self):
        """根据类型创建记忆"""
        if self.memory_type == "buffer":
            return ConversationBufferMemory(return_messages=True, memory_key="history")
        elif self.memory_type == "window":
            return ConversationBufferWindowMemory(k=3, return_messages=True, memory_key="history")
        elif self.memory_type == "summary":
            return ConversationSummaryMemory(llm=self.llm, return_messages=True, memory_key="history")
        elif self.memory_type == "summary_buffer":
            return ConversationSummaryBufferMemory(
                llm=self.llm, max_token_limit=200, return_messages=True, memory_key="history"
            )
        else:
            return ConversationBufferMemory(return_messages=True, memory_key="history")

    def _create_conversation(self):
        """创建对话链"""
        prompt = PromptTemplate(
            input_variables=["history", "input"],
            template="""你是一个专业的客服助手。请根据对话历史和用户问题提供帮助。

对话历史:
{history}

用户问题: {input}

请提供专业、友好的回复。如果涉及具体业务问题，请引导用户提供更多信息。

客服回复:"""
        )

        return ConversationChain(
            llm=self.llm,
            memory=self.memory,
            prompt=prompt,
            verbose=False
        )

    def chat(self, user_input: str) -> str:
        """处理用户输入"""
        # 检查知识库
        for keyword, answer in self.knowledge_base.items():
            if keyword in user_input:
                # 将知识库答案作为上下文
                enhanced_input = f"{user_input}\n\n参考信息: {answer}"
                return self.conversation.predict(input=enhanced_input)

        return self.conversation.predict(input=user_input)

    def get_memory_info(self):
        """获取记忆信息"""
        if hasattr(self.memory, 'chat_memory'):
            return f"记忆类型: {self.memory_type}, 消息数: {len(self.memory.chat_memory.messages)}"
        return f"记忆类型: {self.memory_type}"

def demo_customer_service():
    """演示客服系统"""
    print("\n客服系统演示 (使用SummaryBufferMemory):")

    # 创建客服机器人
    bot = CustomerServiceBot(llm, memory_type="summary_buffer")

    # 模拟客服对话
    customer_queries = [
        "你好，我想咨询一下你们的退货政策",
        "我昨天买了一件衣服，订单号是12345，但是尺码不合适",
        "退货需要什么手续吗？",
        "大概多久能退款到账？",
        "好的，谢谢。另外我想问一下发货时间",
        "我刚下了一个新订单，什么时候能收到？",
        "你还记得我刚才问的退货问题吗？"
    ]

    for i, query in enumerate(customer_queries, 1):
        response = bot.chat(query)
        print(f"\n第{i}轮:")
        print(f"客户: {query}")
        print(f"客服: {response}")
        print(f"记忆状态: {bot.get_memory_info()}")

# 运行客服演示
demo_customer_service()


In [ ]:
# ===================== 9. 最佳实践和性能优化 =====================

print("\n\n9. 最佳实践和性能优化")
print("-" * 35)

def memory_best_practices():
    """记忆使用最佳实践"""

    practices = {
        "选择合适的记忆类型": [
            "短对话(<10轮): ConversationBufferMemory",
            "中等对话(10-50轮): ConversationBufferWindowMemory",
            "长对话(>50轮): ConversationSummaryBufferMemory",
            "严格内存限制: ConversationTokenBufferMemory",
            "需要信息提取: 自定义Memory类"
        ],
        "参数调优建议": [
            "窗口大小: 根据对话特点设置，一般3-10轮",
            "Token限制: 考虑模型上下文长度的50-70%",
            "摘要频率: 平衡性能和信息保留",
            "记忆键名: 使用描述性名称，避免冲突"
        ],
        "性能优化": [
            "异步处理: 使用async/await处理LLM调用",
            "缓存机制: 缓存频繁访问的摘要",
            "批量处理: 批量更新记忆以减少I/O",
            "内存监控: 定期检查内存使用情况"
        ],
        "错误处理": [
            "LLM调用失败: 实现重试机制",
            "记忆溢出: 设置合理的上限和清理策略",
            "数据持久化: 定期保存重要对话",
            "并发安全: 多用户场景下的线程安全"
        ]
    }

    for category, items in practices.items():
        print(f"\n{category}:")
        for item in items:
            print(f"  • {item}")

memory_best_practices()


In [ ]:
# ===================== 10. 记忆持久化演示 =====================

print("\n\n10. 记忆持久化演示")
print("-" * 30)

import json
import pickle
from datetime import datetime

class PersistentMemory:
    """
    支持持久化的记忆管理器
    """

    def __init__(self, memory, session_id: str = None):
        self.memory = memory
        self.session_id = session_id or f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        self.save_path = f"memory_{self.session_id}.json"

    def save_memory(self):
        """保存记忆到文件"""
        try:
            memory_data = {
                "session_id": self.session_id,
                "timestamp": datetime.now().isoformat(),
                "memory_type": type(self.memory).__name__,
                "messages": []
            }

            # 保存消息历史
            if hasattr(self.memory, 'chat_memory'):
                for msg in self.memory.chat_memory.messages:
                    memory_data["messages"].append({
                        "type": type(msg).__name__,
                        "content": msg.content
                    })

            # 保存其他属性
            if hasattr(self.memory, 'moving_summary_buffer'):
                memory_data["summary"] = self.memory.moving_summary_buffer

            with open(self.save_path, 'w', encoding='utf-8') as f:
                json.dump(memory_data, f, ensure_ascii=False, indent=2)

            print(f"记忆已保存到: {self.save_path}")
            return True

        except Exception as e:
            print(f"保存记忆失败: {e}")
            return False

    def load_memory(self, file_path: str = None):
        """从文件加载记忆"""
        try:
            load_path = file_path or self.save_path

            with open(load_path, 'r', encoding='utf-8') as f:
                memory_data = json.load(f)

            print(f"从 {load_path} 加载记忆:")
            print(f"  会话ID: {memory_data['session_id']}")
            print(f"  时间戳: {memory_data['timestamp']}")
            print(f"  记忆类型: {memory_data['memory_type']}")
            print(f"  消息数量: {len(memory_data['messages'])}")

            # 重建消息历史
            if hasattr(self.memory, 'chat_memory'):
                self.memory.chat_memory.clear()
                for msg_data in memory_data["messages"]:
                    if msg_data["type"] == "HumanMessage":
                        self.memory.chat_memory.add_user_message(msg_data["content"])
                    elif msg_data["type"] == "AIMessage":
                        self.memory.chat_memory.add_ai_message(msg_data["content"])

            # 恢复摘要
            if "summary" in memory_data and hasattr(self.memory, 'moving_summary_buffer'):
                self.memory.moving_summary_buffer = memory_data["summary"]

            return True

        except Exception as e:
            print(f"加载记忆失败: {e}")
            return False

def demo_persistent_memory():
    """演示记忆持久化"""
    print("\n记忆持久化演示:")

    # 创建一个新的记忆实例
    memory = ConversationSummaryBufferMemory(
        llm=llm,
        max_token_limit=100,
        return_messages=True,
        memory_key="history"
    )

    # 创建持久化管理器
    persistent_mgr = PersistentMemory(memory, "demo_session")

    # 模拟一些对话
    conversation = ConversationChain(llm=llm, memory=memory, verbose=False)

    print("\n模拟对话...")
    test_inputs = [
        "我是测试用户，正在学习LangChain",
        "我对记忆机制特别感兴趣",
        "希望能在实际项目中应用这些技术"
    ]

    for user_input in test_inputs:
        response = conversation.predict(input=user_input)
        print(f"用户: {user_input}")
        print(f"AI: {response[:100]}...")  # 只显示前100个字符

    # 保存记忆
    print(f"\n保存记忆...")
    persistent_mgr.save_memory()

    # 创建新的记忆实例并加载
    print(f"\n创建新记忆实例并加载...")
    new_memory = ConversationSummaryBufferMemory(
        llm=llm,
        max_token_limit=100,
        return_messages=True,
        memory_key="history"
    )

    new_persistent_mgr = PersistentMemory(new_memory, "demo_session")
    if new_persistent_mgr.load_memory():
        print("记忆加载成功！")

        # 测试加载的记忆
        new_conversation = ConversationChain(llm=llm, memory=new_memory, verbose=False)
        test_response = new_conversation.predict(input="你还记得我之前说过什么吗？")
        print(f"\n测试记忆恢复:")
        print(f"用户: 你还记得我之前说过什么吗？")
        print(f"AI: {test_response}")

# 运行持久化演示
demo_persistent_memory()


In [ ]:
# ===================== 总结 =====================

print("\n\n" + "="*60)
print("LangChain 记忆对话系统演示总结")
print("="*60)

summary_text = """
本演示展示了LangChain中的各种记忆机制：

1. ConversationBufferMemory
   - 最简单的记忆类型，保存完整对话历史
   - 适合短对话和内存充足的场景

2. ConversationBufferWindowMemory
   - 只保留最近N轮对话
   - 内存使用固定，适合中等长度对话

3. ConversationSummaryMemory
   - 将历史对话总结为摘要
   - 适合超长对话，但可能丢失细节

4. ConversationSummaryBufferMemory
   - 结合摘要和缓冲的优点
   - 生产环境推荐，平衡性能和信息保留

5. ConversationTokenBufferMemory
   - 基于token数量的精确控制
   - 适合API成本敏感的场景

6. 自定义记忆类
   - 可以实现特定的业务逻辑
   - 如用户信息提取、情感分析等

7. 实际应用示例
   - 智能客服系统
   - 记忆持久化
   - 性能优化策略

选择建议：
- 开发阶段：使用ConversationBufferMemory进行快速原型
- 生产环境：推荐ConversationSummaryBufferMemory
- 特殊需求：考虑自定义记忆类

记住：记忆机制的选择应该基于具体的应用场景、性能要求和资源限制。
"""

print(summary_text)

print("\n演示完成！")
print("你可以根据需要修改参数，尝试不同的记忆配置。")
print("建议在实际项目中进行充分测试，找到最适合的记忆策略。")
